In [6]:
from pyiron_workflow import Workflow
from structure_nodes import get_structure
from thermo_potential import obtain_potential
import numpy as np

In [12]:
wf.run()

{'step1__config': ['pair_style eam/alloy\n',
  'pair_coeff * * CuAgAu_Zhou04.eam.alloy Cu Ag Au\n'],
 'step1__files': ['potential_LAMMPS/2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2/CuAgAu_Zhou04.eam.alloy'],
 'step1__elements': ['Cu', 'Ag', 'Au'],
 'step1__element_list':     Parameter                                 Value Comment
 0  pair_style                             eam/alloy        
 1  pair_coeff  * * CuAgAu_Zhou04.eam.alloy Cu Ag Au        ,
 'step2__structure': Cu: [0. 0. 0.]
 Cu: [0.     1.7985 1.7985]
 Cu: [1.7985 0.     1.7985]
 Cu: [1.7985 1.7985 0.    ]
 Cu: [0.    0.    3.597]
 Cu: [0.     1.7985 5.3955]
 Cu: [1.7985 0.     5.3955]
 Cu: [1.7985 1.7985 3.597 ]
 Cu: [0.    3.597 0.   ]
 Cu: [0.     5.3955 1.7985]
 Cu: [1.7985 3.597  1.7985]
 Cu: [1.7985 5.3955 0.    ]
 Cu: [0.    3.597 3.597]
 Cu: [0.     5.3955 5.3955]
 Cu: [1.7985 3.597  5.3955]
 Cu: [1.7985 5.3955 3.597 ]
 Cu: [3.597 0.    0.   ]
 Cu: [3.597  1.7985 1.7985]
 Cu: [5.3955 0.     1.7985]
 Cu: [5.3955 1.7985 0.

In [20]:
wf.outputs

In [ ]:
wf.outputs.step1__

In [16]:
wf.outputs.step2__structure.value.plot3d()

NGLWidget()

### Create input template

In [17]:
def CreateInputDict():
    return {
        "mode": None,
        "pressure": 0,
        "temperature": 0,
        "reference_phase": None,
        "npt": True,
        "n_equilibration_steps": 15000,
        "n_switching_steps": 25000,
        "n_print_steps": 1000,
        "n_iterations": 1,
        "spring_constants": None,
        "equilibration_control": "nose-hoover",
        "melting_cycle": True,
        "md": {
            "timestep": 0.001,
            "n_small_steps": 10000,
            "n_every_steps": 10,
            "n_repeat_steps": 10,
            "n_cycles": 100,
            "thermostat_damping": 0.5,
            "barostat_damping": 0.1,
        },
        "tolerance": {
            "lattice_constant": 0.0002,
            "spring_constant": 0.01,
            "solid_fraction": 0.7,
            "liquid_fraction": 0.05,
            "pressure": 0.5,
        },
        "nose_hoover": {
            "thermostat_damping": 0.1,
            "barostat_damping": 0.1,
        },
        "berendsen": {
            "thermostat_damping": 100.0,
            "barostat_damping": 100.0,
        },
    }

In [18]:
inp = CreateInputDict()

In [19]:
inp['temperature'] = 1000
inp['pressure'] = 0
inp['reference_phase'] = 'solid'
inp['mode'] = 'fe'

In [26]:
wf.run()

/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-02/lib/python3.11/site-packages/ase/io/lammpsdata.py:49: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(


{'step1__elements': ['Cu', 'Ag', 'Au'],
 'step3__calculation': Calculation(monte_carlo=MonteCarlo(n_steps=1, n_swaps=0, reverse_swap=True), composition_scaling=CompositionScaling(output_chemical_composition={}, restrictions=[]), md=MD(timestep=0.001, n_small_steps=10000, n_every_steps=10, n_repeat_steps=10, n_cycles=100, thermostat_damping=0.5, barostat_damping=0.1, cmdargs='', init_commands=[]), nose_hoover=NoseHoover(thermostat_damping=0.1, barostat_damping=0.1), berendsen=Berendsen(thermostat_damping=100.0, barostat_damping=100.0), queue=Queue(scheduler='local', cores=1, jobname='calphy', walltime='23:59:00', queuename='', memory='3GB', commands=[], options=[], modules=[]), tolerance=Tolerance(lattice_constant=0.0002, spring_constant=0.01, solid_fraction=0.7, liquid_fraction=0.05, pressure=0.5), melting_temperature=MeltingTemperature(guess=None, step=200, attempts=5), element=['Cu', 'Ag', 'Au'], n_elements=3, mass=[63.546, 1.0, 1.0], kernel=0, inputfile='', mode='fe', lattice='/cmmc

### Initialise job

In [43]:
@Workflow.wrap.as_function_node()
def Initialize(inputdict, structure, potential_obj, files, potential_config):
    from pyiron_atomistics.lammps.structure import (
        LammpsStructure,
        UnfoldingPrism,
        structure_to_lammps,
    ) 
    import os
    from calphy.input import Calculation
    import shutil
    
    #filename of structure
    file_name = os.path.join(os.getcwd(), 'temp.struct.dat')
    lmp_structure = LammpsStructure()
    lmp_structure.potential = potential_obj
    lmp_structure.atom_type = "atomic"
    lmp_structure.el_eam_lst = potential_obj.get_element_lst()
    lmp_structure.structure = structure_to_lammps(structure)
    if not set(lmp_structure.structure.get_species_symbols()).issubset(
        set(lmp_structure.el_eam_lst)
    ):
        raise ValueError(
            "The selected potentials do not support the given combination of elements."
        )
    lmp_structure.write_file(file_name=file_name)

    #now populate the input dict
    inputdict['lattice'] = file_name
    inputdict['pair_style'] = potential_config[0]
    inputdict['pair_coeff'] = potential_config[1]
    inputdict['element'] = potential_obj.get_element_lst()

    elements_from_pot = inputdict['element']
    elements_object_lst = structure.get_species_objects()
    elements_struct_lst = structure.get_species_symbols()

    masses = []
    for element_name in elements_from_pot:
        if element_name in elements_struct_lst:
            index = list(elements_struct_lst).index(element_name)
            masses.append(elements_object_lst[index].AtomicMass)
        else:
            masses.append(1.0)
 
    inputdict['mass'] = masses
    
    #working directory
    calculation = Calculation(**inputdict)
    simfolder = calculation.create_folders()
    potential_obj.copy_pot_files(simfolder)
    #for file in files:
    #    shutil.copy(file, simfolder)
        
    return calculation, simfolder

In [52]:
@Workflow.wrap.as_function_node()
def RunCalculation(calculation, simfolder):
    from calphy import Liquid, Solid
    from calphy.routines import routine_alchemy, routine_fe, routine_pscale, routine_ts
    #simfolder = calc.create_folders()
    if calculation.reference_phase == "solid":
        job = Solid(calculation=calculation, simfolder=simfolder)
    elif calculation.reference_phase == "liquid":
        job = Liquid(calculation=calculation, simfolder=simfolder)
    else:
        raise ValueError("Unknown reference state")

    if calculation.mode == "fe":
        routine_fe(job)
    elif calculation.mode == "ts":
        routine_ts(job)
    elif calculation.mode == "pscale":
        routine_pscale(job)
    else:
        raise ValueError("Unknown mode")
    return job.report

In [53]:
wf = Workflow('test')
wf.step1 = obtain_potential('2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2')
wf.step2 = get_structure(a0=3.597, species='Cu', 
                         cubic=True, repeat=np.array([2,2,2]),
                        lattice_type='fcc',
                        a2=None)
wf.step3 = Initialize(inp, wf.step2.outputs.structure,
                     wf.step1.outputs.element_list,
                     wf.step1.outputs.files,
                     wf.step1.outputs.config)
wf.step4 = RunCalculation(wf.step3.outputs.calculation,
                         wf.step3.outputs.simfolder)

In [ ]:
wf.run()

/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-02/lib/python3.11/site-packages/ase/io/lammpsdata.py:49: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(
--------------------------------------------------------------------------

  Local host:   cmti001
  Local device: hfi1_0
--------------------------------------------------------------------------
Traceback (most recent call last):
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-02/lib/python3.11/site-packages/pylammpsmpi/wrapper/../mpi/lmpmpi.py", line 503, in <module>
    _run_lammps_mpi(argument_lst=sys.argv)
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-02/lib/python3.11/site-packages/pylammpsmpi/wrapper/../mpi/lmpmpi.py", line 495, in _run_lammps_mpi
    output = select_cmd(input_dict["command"])(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-02/lib/python3.11/site-package

In [ ]:
calculations:
- temperature: 1000
  element:
  - Au
  - Cu
  mass: 
  - 196.966
  - 63.546
  composition_scaling:
    output_chemical_composition:
      Au: 1024
      Cu: 1024
  lattice: 'lammps.inp.data'
  pair_style: eam/alloy
  pair_coeff: "* * /u/smenon/calphy_configurational_entropy/potential/CuAu.Zhou.eam.alloy Au Cu"
  reference_phase: solid
  md:
    timestep: 0.001
  mode: composition_scaling
  n_equilibration_steps: 25000
  n_iterations: 1
  n_switching_steps: 50000
  pressure: [[0,0,0]]
  queue:
    cores: 40
    queuename: p.cmfe
    scheduler: slurm
